In [16]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

client = openai.Client()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [17]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)

In [18]:
print(file.id)

file-Y9GcTekRZnshT9NTQdzv6G


In [19]:
assistant = client.beta.assistants.create(
    name="Anaista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas.",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
    model="gpt-4o"
)

In [20]:
# pergunta = "Qual rating médio das vendas do supermercado?"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"

In [21]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [22]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [24]:
# Aguarda a thread ser executada
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [25]:
# Verificar resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro: {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_zyHSCHjkoJAsDrOJwjLqVB3z', assistant_id='asst_HJn3Aszj4LGQ8ny2W7gNnfMH', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-QL6mi99SwvFUKfyYHEByGf', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza que mostra o percentual de vendas por linha de produto. Cada segmento do gráfico representa a proporção de vendas de uma categoria de produto específica em relação ao total de vendas. Se precisar de mais alguma coisa, é só me avisar!'), type='text')], created_at=1747272341, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ExfnSj7qT0qV66yJoAOSdkpS', status=None, thread_id='thread_CkEPpxZXumhIyZbqhVzL33I6'), Message(id='msg_JIn4JMWhjoLiYvk9AnshTmYg', assistant_id='asst_HJn3Aszj4LGQ8ny2W7gNnfMH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotati

In [31]:
mensagens.data[0].content[0].text.value

AttributeError: 'ImageFileContentBlock' object has no attribute 'text'

In [27]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [ ]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_cals":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        # print(message.content[0].text.value)


====Step message_creation
Primeiro, vou examinar o conteúdo do arquivo para entender sua estrutura e identificar os dados necessários para gerar o gráfico de pizza.

====Step tool_calls

====Step message_creation
Ocorreu um erro ao tentar carregar o arquivo. É possível que o arquivo não seja um documento do Excel ou que o formato dele não tenha sido reconhecido automaticamente. Vou tentar carregar o arquivo novamente, especificando manualmente os tipos de engine possíveis para leitura de Excel, como `openpyxl` ou `xlrd`. Vamos ver se isso resolve o problema.

====Step tool_calls

====Step message_creation
Parece que o arquivo não é um documento do Excel válido, ou ele pode ter sido salvo com uma extensão incorreta. Vamos tentar abri-lo como um arquivo CSV, que é outro formato comum para dados tabulares.

====Step tool_calls

====Step message_creation
O arquivo foi carregado com sucesso como um arquivo CSV. Agora, podemos observar que ele contém uma coluna chamada "Product line", que p